In [74]:
import cv2
recognizer = cv2.face.LBPHFaceRecognizer_create()
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import ggmm.gpu as ggmm

In [75]:
import sys, glob, argparse
import numpy as np
import math
from scipy.stats import multivariate_normal
import time
from sklearn import svm

num_of_images = 10000
# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

def image_vgg_desc(file):
    data = sio.loadmat(file)
    return data['features']

def image_vgg_desc_averaged(file, n):
    data = sio.loadmat(file)
    data = data['features']
    avg_data = [sum(data[0][i:i+n])//n for i in range(0,len(data[0]),n)]
    avg_data = np.array(avg_data).reshape((1, len(avg_data)))
    #print('data shape - ', avg_data.shape)
    return avg_data

def folder_descriptors(folder):
    files = sorted(glob.glob(folder + "/*.mat"))
    files = files[:num_of_images]
    print("Calculating descriptos. Number of images is", len(files))
    return np.concatenate([image_vgg_desc_averaged(file, 4) for file in files])


def generate_gmm(input_folder):
    words = np.concatenate([folder_descriptors(input_folder)]) 
    print("Training GMM of size, input size - ", words.shape)
    
    N, D = words.shape
    K = 32 #128 #256 #4096 #128

    ggmm.init()
    gmm = ggmm.GMM(K,D)

    thresh = 1e-3 # convergence threshold
    n_iter = 20 # maximum number of EM iterations
    init_params = 'wmc' # initialize weights, means, and covariances

    # train GMM
    gmm.fit(words, thresh, n_iter, init_params=init_params)
    
    weights = gmm.get_weights()
    means = gmm.get_means()
    covs = gmm.get_covars()

    print("dict done")

    np.save("means_100.gmm", means)
    np.save("covs_100.gmm", covs)
    np.save("weights_100.gmm", weights)
    print("GMM is done")
    return means, covs, weights

GPU is active


In [76]:


m, c, w = generate_gmm('gen_data/vgg_resize/')

Calculating descriptos. Number of images is 10000
Training GMM of size, input size -  (10000, 1024)
dict done
GMM is done


In [77]:


def likelihood_moment(x, ytk, moment):
    x_moment = np.power(np.float32(x), moment) if moment > 0 else np.float32([1])
    return x_moment * ytk

def likelihood_statistics(samples, means, covs, weights):
    gaussians, s0, s1,s2 = {}, {}, {}, {}
    #print(samples.shape)
    samples = zip(range(0, len(samples)), samples)

    g = [multivariate_normal(mean=means[k], cov=covs[k]) for k in range(0, len(weights)) ]
    for index, x in samples:
        gaussians[index] = np.array([g_k.pdf(x) for g_k in g])

    for k in range(0, len(weights)):
        s0[k], s1[k], s2[k] = 0, 0, 0
        for index, x in samples:
            probabilities = np.multiply(gaussians[index], weights)
            probabilities = probabilities / np.sum(probabilities)
            s0[k] = s0[k] + likelihood_moment(x, probabilities[k], 0)
            s1[k] = s1[k] + likelihood_moment(x, probabilities[k], 1)
            s2[k] = s2[k] + likelihood_moment(x, probabilities[k], 2)

    return s0, s1, s2

def fisher_vector_weights(s0, s1, s2, means, covs, w, T):
    return np.float32([((s0[k] - T * w[k]) / np.sqrt(w[k]) ) for k in range(0, len(w))])

def fisher_vector_means(s0, s1, s2, means, sigma, w, T):
    #print(len(s0), len(s1), len(s2), len(w))
    #print(means[0].shape, sigma[0].shape)
    a = [(s1[k] - means[k][:256] * s0[k]) for k in range(0, len(w))]
    b = [(np.sqrt(w[k] * sigma[0][k])) for k in range(0, len(w))]
    #return np.float32(a/b)
    return np.float32([(s1[k] - means[k][:256] * s0[k]) / (np.sqrt(w[k] * sigma[0][k])) for k in range(0, len(w))])

def fisher_vector_sigma(s0, s1, s2, means, sigma, w, T):
    return np.float32([(s2[k] - 2 * means[k][:256]*s1[k]  + (means[k][:256]*means[k][:256] - sigma[0][k]) * s0[k]) 
                       / (np.sqrt(2*w[k])*sigma[0][k])  for k in range(0, len(w))])

def normalize(fisher_vector):
    v = np.sqrt(abs(fisher_vector)) * np.sign(fisher_vector)
    #print('in normalize')
    return v / np.sqrt(np.dot(v, v))

def fisher_vector(samples, means, covs, w):
    #return 0
    s0, s1, s2 =  likelihood_statistics(samples, means, covs, w)
    T = samples.shape[0]
    #print('in fisher _vector likelihood done')
    #covs = np.float32([np.diagonal(covs[k]) for k in range(0, covs.shape[0])])
    covs = np.float32([np.diagonal(covs)])
    a = fisher_vector_weights(s0, s1, s2, means, covs, w, T)
    b = fisher_vector_means(s0, s1, s2, means, covs, w, T)
    c = fisher_vector_sigma(s0, s1, s2, means, covs, w, T)
    #print('outputs -', a.shape, b[0].shape, c.shape)
    #fv = np.concatenate([np.concatenate(a), np.concatenate(b[0]), np.concatenate(c[0])])
    fv = np.concatenate([a, b[0], c[0]])
    fv = normalize(fv)
    print('image - ', time.time())
    #img_count = img_count + 1
    return fv

"""
def get_fisher_vectors_from_folder(folder, means, covs, w):
    files = glob.glob(folder + "/*.mat")
    files = files[:num_of_images]
    #print (files)
    return np.float32([fisher_vector(image_vgg_desc_averaged(file, 4), means, covs, w) for file in files])
"""
def fisher_features(folder, means, covs, w):
    outpath = 'gen_data/fv_1024/'
    print('means shape - ', means.shape)
    print('covs shape - ', covs.shape)
    print('weights shape - ', w.shape)
    files = sorted(glob.glob(folder + "/*.mat"))
    files = files[:num_of_images]
    
    count = 1
    for file in files:
        fv = np.float32([fisher_vector(image_vgg_desc_averaged(file, 4), means, covs, w)])
        f = file.split('/')
        f = f[2].split('.')
        print("saving the file ", f[0] + '.fv', "count - ", count)
        count = count + 1
        sio.savemat(outpath + f[0] +'.fv', {'fv':fv}, do_compression=True)
    #features = {'vgg_resize' : get_fisher_vectors_from_folder(folder, means, covs, w)}

    return

In [ ]:
t0 = time.time()
fisher_features('gen_data/vgg_resize/', m, c, w)
print('fisher_features are generated')
print("Extracting training fisher vectors for 10000 images took %0.3fs." % (time.time() - t0))

means shape -  (32, 1024)
covs shape -  (32, 1024)
weights shape -  (32,)
image -  1544935592.3594599
saving the file  10.fv count -  1
image -  1544935594.4979692
saving the file  100009.fv count -  2
image -  1544935596.653287
saving the file  100021.fv count -  3
image -  1544935598.7796898
saving the file  100032.fv count -  4
image -  1544935600.8934908
saving the file  100034.fv count -  5
image -  1544935602.9902275
saving the file  100036.fv count -  6
image -  1544935605.181097
saving the file  100039.fv count -  7
image -  1544935607.3120868
saving the file  10004.fv count -  8
image -  1544935609.3956997
saving the file  100043.fv count -  9
image -  1544935611.4673505
saving the file  100048.fv count -  10
image -  1544935613.546277
saving the file  100061.fv count -  11
image -  1544935615.5995605
saving the file  100073.fv count -  12
image -  1544935617.6488738
saving the file  100087.fv count -  13
image -  1544935619.753039
saving the file  100088.fv count -  14
image 

image -  1544935850.2928772
saving the file  100701.fv count -  123
image -  1544935852.3983605
saving the file  100718.fv count -  124
image -  1544935854.4275613
saving the file  100721.fv count -  125
image -  1544935856.465805
saving the file  100745.fv count -  126
image -  1544935858.5186605
saving the file  100750.fv count -  127
image -  1544935860.6220372
saving the file  100758.fv count -  128
image -  1544935862.6562111
saving the file  100760.fv count -  129
image -  1544935864.6617815
saving the file  100767.fv count -  130
image -  1544935866.6804626
saving the file  100771.fv count -  131
image -  1544935868.7070432
saving the file  100773.fv count -  132
image -  1544935870.7419674
saving the file  100776.fv count -  133
image -  1544935872.7922122
saving the file  100779.fv count -  134
image -  1544935874.8243763
saving the file  100784.fv count -  135
image -  1544935876.7956254
saving the file  100790.fv count -  136
image -  1544935878.7857964
saving the file  1008

image -  1544936102.6072226
saving the file  101385.fv count -  245
image -  1544936104.7477496
saving the file  101392.fv count -  246
image -  1544936106.8450282
saving the file  101411.fv count -  247
image -  1544936108.9669118
saving the file  101420.fv count -  248
image -  1544936111.0618007
saving the file  101423.fv count -  249
image -  1544936113.2479181
saving the file  101429.fv count -  250
image -  1544936115.3641896
saving the file  101432.fv count -  251
image -  1544936117.490488
saving the file  101435.fv count -  252
image -  1544936119.6069057
saving the file  101436.fv count -  253
image -  1544936121.7097504
saving the file  101440.fv count -  254
image -  1544936123.8172367
saving the file  101444.fv count -  255
image -  1544936125.9280374
saving the file  101447.fv count -  256
image -  1544936128.0502002
saving the file  101451.fv count -  257
image -  1544936130.138981
saving the file  101455.fv count -  258
image -  1544936132.238331
saving the file  101458

image -  1544936358.6215844
saving the file  102130.fv count -  367
image -  1544936360.714611
saving the file  102131.fv count -  368
image -  1544936362.8377075
saving the file  102134.fv count -  369
image -  1544936364.9715507
saving the file  102138.fv count -  370
image -  1544936367.1196349
saving the file  102140.fv count -  371
image -  1544936369.209083
saving the file  102149.fv count -  372
image -  1544936371.2782776
saving the file  102151.fv count -  373
image -  1544936373.3536096
saving the file  102154.fv count -  374
image -  1544936375.4394588
saving the file  10216.fv count -  375
image -  1544936377.4156625
saving the file  102170.fv count -  376
image -  1544936379.4548903
saving the file  102171.fv count -  377
image -  1544936381.565398
saving the file  102176.fv count -  378
image -  1544936383.5817804
saving the file  10218.fv count -  379
image -  1544936385.6296043
saving the file  102180.fv count -  380
image -  1544936387.658139
saving the file  102187.fv

image -  1544936614.9115226
saving the file  102845.fv count -  489
image -  1544936616.9176376
saving the file  102849.fv count -  490
image -  1544936618.981827
saving the file  102853.fv count -  491
image -  1544936621.0975223
saving the file  102858.fv count -  492
image -  1544936623.220067
saving the file  102863.fv count -  493
image -  1544936625.3661437
saving the file  102870.fv count -  494
image -  1544936627.4789262
saving the file  102871.fv count -  495
image -  1544936629.5384834
saving the file  102883.fv count -  496
image -  1544936631.5835304
saving the file  102886.fv count -  497
image -  1544936633.7584388
saving the file  102888.fv count -  498
image -  1544936635.9807508
saving the file  10289.fv count -  499
image -  1544936637.991294
saving the file  102896.fv count -  500
image -  1544936640.0637271
saving the file  102904.fv count -  501
image -  1544936642.1720433
saving the file  102905.fv count -  502
image -  1544936644.249355
saving the file  102913.f

...

image -  1544939562.6102161
saving the file  111074.fv count -  1936
image -  1544939564.7290292
saving the file  111080.fv count -  1937
image -  1544939566.4728873
saving the file  111095.fv count -  1938
image -  1544939568.2072937
saving the file  111098.fv count -  1939
image -  1544939569.9954295
saving the file  111113.fv count -  1940
image -  1544939571.781033
saving the file  111115.fv count -  1941
image -  1544939573.4568741
saving the file  111117.fv count -  1942
image -  1544939575.0770068
saving the file  11112.fv count -  1943
image -  1544939577.0624743
saving the file  111121.fv count -  1944
image -  1544939579.1220667
saving the file  111123.fv count -  1945
image -  1544939581.1630838
saving the file  111127.fv count -  1946
image -  1544939583.1856346
saving the file  111131.fv count -  1947
image -  1544939585.203022
saving the file  111139.fv count -  1948
image -  1544939587.2619498
saving the file  11114.fv count -  1949
image -  1544939589.3239193
saving the

image -  1544939806.7866502
saving the file  111719.fv count -  2056
image -  1544939808.8092039
saving the file  111721.fv count -  2057
image -  1544939810.8911462
saving the file  111726.fv count -  2058
image -  1544939812.911795
saving the file  111733.fv count -  2059
image -  1544939814.89969
saving the file  111739.fv count -  2060
image -  1544939816.8936837
saving the file  111747.fv count -  2061
image -  1544939818.987441
saving the file  111753.fv count -  2062
image -  1544939821.152781
saving the file  111755.fv count -  2063
image -  1544939823.3192527
saving the file  111759.fv count -  2064
image -  1544939825.4964027
saving the file  111761.fv count -  2065
image -  1544939827.6641233
saving the file  111768.fv count -  2066
image -  1544939829.8342497
saving the file  111769.fv count -  2067
image -  1544939831.8695319
saving the file  11177.fv count -  2068
image -  1544939833.914304
saving the file  111775.fv count -  2069
image -  1544939835.9547622
saving the fi

image -  1544940054.9977944
saving the file  112408.fv count -  2176
image -  1544940057.0897303
saving the file  112409.fv count -  2177
image -  1544940059.1868293
saving the file  112416.fv count -  2178
image -  1544940061.306678
saving the file  112417.fv count -  2179
image -  1544940063.4641554
saving the file  112423.fv count -  2180
image -  1544940065.5345433
saving the file  112426.fv count -  2181
image -  1544940067.5595162
saving the file  112439.fv count -  2182
image -  1544940069.6202161
saving the file  11244.fv count -  2183
image -  1544940071.8062198
saving the file  112441.fv count -  2184
image -  1544940074.060706
saving the file  11247.fv count -  2185
image -  1544940076.1680796
saving the file  112479.fv count -  2186
image -  1544940078.2911894
saving the file  11249.fv count -  2187
image -  1544940080.5217404
saving the file  112491.fv count -  2188
image -  1544940082.621312
saving the file  112493.fv count -  2189
image -  1544940084.715641
saving the fi

image -  1544940297.3329625
saving the file  113086.fv count -  2296
image -  1544940299.393813
saving the file  113093.fv count -  2297
image -  1544940301.5100307
saving the file  113100.fv count -  2298
image -  1544940303.6858783
saving the file  113106.fv count -  2299
image -  1544940305.901945
saving the file  11311.fv count -  2300
image -  1544940308.1172225
saving the file  113115.fv count -  2301
image -  1544940310.2629259
saving the file  113119.fv count -  2302
image -  1544940312.4288814
saving the file  113134.fv count -  2303
image -  1544940314.5303936
saving the file  113141.fv count -  2304
image -  1544940316.5354738
saving the file  113148.fv count -  2305
image -  1544940318.612476
saving the file  113149.fv count -  2306
image -  1544940320.6960604
saving the file  113160.fv count -  2307
image -  1544940322.775964
saving the file  113170.fv count -  2308
image -  1544940324.8806474
saving the file  113173.fv count -  2309
image -  1544940326.9070055
saving the 

image -  1544940569.6900458
saving the file  113809.fv count -  2427
image -  1544940571.800119
saving the file  113811.fv count -  2428
image -  1544940573.8972394
saving the file  113814.fv count -  2429
image -  1544940575.9725137
saving the file  113818.fv count -  2430
image -  1544940578.0522518
saving the file  11382.fv count -  2431
image -  1544940580.1132658
saving the file  113826.fv count -  2432
image -  1544940582.1315992
saving the file  113856.fv count -  2433
image -  1544940584.3277214
saving the file  113864.fv count -  2434
image -  1544940586.5409682
saving the file  113865.fv count -  2435
image -  1544940588.7040615
saving the file  113866.fv count -  2436
image -  1544940590.9111922
saving the file  113868.fv count -  2437
image -  1544940593.0958636
saving the file  113872.fv count -  2438
image -  1544940595.3174164
saving the file  113879.fv count -  2439
image -  1544940597.528464
saving the file  113884.fv count -  2440
image -  1544940599.733865
saving the

In [ ]:
#CSV file reading
"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))